# Data Integration with CHAKRA

## Import CHAKRA library

In [1]:
using Chakra

## Implement an interface for a data source S1

The interface must define the core CHAKRA types and operations.

In [2]:
module S1
    using Chakra, DataStructures
    include("Ref.jl")
    @REF Int []
    
    # DEFINE ATTRIBUTES

    Chakra.@Attribute(:A,Int)
    Chakra.@Attribute(:B,String)

    o = Chakra.seta(Att(:A),10,c(Id[]))

    data = Chakra.ins(Id(1),o,emp(Hierarchy))

end

LoadError: UndefVarError: setp not defined

In [20]:
typeof(S1.data)

Main.S1.Hierarchy

In [21]:
dom(S1.data)

1-element Vector{Main.S1.Id}:
 Main.S1.Id(1)

In [22]:
geta(:A,S1.Id(1),S1.data)

10

## Implement a second data source interface S2

In [23]:
module S2
    using Chakra, DataStructures
    include("Ref.jl")
    @REF Int []

    data = Chakra.ins!(Id(2),c(Id[]),emp(Hierarchy))

end

Main.S2

In [24]:
dom(S2.data)

1-element Vector{Main.S2.Id}:
 Main.S2.Id(2)

## Link data sources S1 and S2 using a third data source S3

In [25]:
[m.Id for m in [S1, S2]]

2-element Vector{DataType}:
 Main.S1.Id
 Main.S2.Id

In [27]:
module S3
    using Chakra, DataStructures
    using Main.S1, Main.S2
    
    include("Ref.jl")
    @REF Int [S1,S2]
    
    Chakra.fnd(x::S1.Id,h::Hierarchy) = fnd(x,S1.data)
    Chakra.fnd(x::S2.Id,h::Hierarchy) = fnd(x,S2.data)
    Chakra.dom(h::Hierarchy) = vcat(collect(keys(h.constituents)),dom(S1.data),dom(S2.data))

    data = Chakra.ins(Id(3),c(DOMAIN[S1.Id(1),S2.Id(2)]),emp(Hierarchy))

end

Main.S3

In [28]:
dom(S3.data)

3-element Vector{Any}:
 Main.S3.Id(3)
 Main.S1.Id(1)
 Main.S2.Id(2)

In [29]:
o = fnd(S3.Id(3),S3.data)

Main.S3.Constituent(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Union{Main.S1.Id, Main.S2.Id, Main.S3.Id}[Main.S1.Id(1), Main.S2.Id(2)])

In [30]:
l = pts(o)

2-element Vector{Union{Main.S1.Id, Main.S2.Id, Main.S3.Id}}:
 Main.S1.Id(1)
 Main.S2.Id(2)

In [32]:
sequence(S3.Id(3),S3.data)

2-element Vector{Any}:
 Main.S1.Constituent(Dict{Symbol, Any}(:A => 10), Dict{Symbol, Any}(), Main.S1.Id[])
 Main.S2.Constituent(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Main.S2.Id[])

In [33]:
sequence(l,S3.data)

2-element Vector{Any}:
 Main.S1.Constituent(Dict{Symbol, Any}(:A => 10), Dict{Symbol, Any}(), Main.S1.Id[])
 Main.S2.Constituent(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Main.S2.Id[])

## Example using JSON data

In [14]:
module CWISA95 

using JSON, Chakra


function load_file()
    d = nothing
    open("cwisa95.json","r") do f
        txt = read(f,String)
        d = JSON.parse(txt)
    end
    return d
end

json = load_file()


struct Id
    id::Symbol
end

cwisa95 = Id(:cwisa95)
step0 = Id(:step0)
step1 = Id(:step1)

abstract type Constituent end
struct Step <: Constituent 
    index::Int
end

struct Directive <: Constituent end

struct Hierarchy end

Chakra.@Attribute(:description,String)


Chakra.pts(c::Step) = Constituent[]
Chakra.pts(::Directive) = Step[step0,step1]

Chakra.fnd(::Val{:cwisa95},::Hierarchy) = Directive()
Chakra.fnd(::Val{:step0},::Hierarchy) = Step(0) 
Chakra.fnd(::Val{:step1},::Hierarchy) = Step(1)
Chakra.fnd(x::Id,h::Hierarchy) = fnd(Val(x.id),h)

Chakra.geta(::Att{a,T},c::Step) where {a,T} = json["workDirective"][c.index+1][String(a)]

Chakra.dom(::Hierarchy) = [cwisa95,step0,step1]

data = Hierarchy()

end


CWISA95.data

Main.CWISA95.Hierarchy()

In [15]:
dom(CWISA95.data)

3-element Vector{Main.CWISA95.Id}:
 Main.CWISA95.Id(:cwisa95)
 Main.CWISA95.Id(:step0)
 Main.CWISA95.Id(:step1)

In [16]:
c = fnd(CWISA95.step0,CWISA95.data)

Main.CWISA95.Step(0)

In [17]:
geta(:description,c)

"Place the bottom plate on the assembly station"

## Next Steps

1. Generate interfaces for AI assisted operator data sources
2. Describe these data sources using the CHAKRA description language

In [18]:
abstract type READ{T} end
abstract type STATE{T} end

abstract type PROP end
abstract type PROOF end

struct Eq{T} <: PROP
    x::T 
    y::T
end

struct Refl{T} <: PROOF
    x::T
end

gettype(x::Refl{T}) where T = Eq(x.x,x.x)

struct CONJ <: PROP
    left::PROP
    right::PROP
end

struct Pair <: PROOF
    left::PROOF
    right::PROOF
end

gettype(x::Pair) = CONJ(gettype(x.left),gettype(x.right))

abstract type HPROP end

struct HLIFT <: HPROP
    p::PROP
end

(h::HLIFT)(s)::P where P<: PROP = h.p